# Syntheseus 多步逆合成完整教程

本教程将全面展示如何使用 Syntheseus 进行多步逆合成规划，包括：
- ⚙️ 单步反应模型的选择和配置
- 🧪 原料库的设置和管理
- 🔍 搜索算法的选择和参数调优
- 📊 结果评测和分析
- 🎨 可视化展示

## 目录
1. [环境准备与导入](#1-环境准备与导入)
2. [单步反应模型](#2-单步反应模型)
3. [原料库配置](#3-原料库配置)
4. [搜索算法配置](#4-搜索算法配置)
5. [运行多步搜索](#5-运行多步搜索)
6. [结果评测与分析](#6-结果评测与分析)
7. [高级功能](#7-高级功能)

## 1. 环境准备与导入

In [ ]:
# 导入必要的库
from syntheseus import Molecule
from syntheseus.reaction_prediction.inference import (
    LocalRetroModel,
    ChemformerModel,
    MEGANModel,
    MHNreactModel,
    RetroKNNModel,
)
from syntheseus.search.mol_inventory import SmilesListInventory
from syntheseus.search.algorithms.best_first.retro_star import RetroStarSearch
from syntheseus.search.algorithms.mcts.molset import MolSetMCTS
from syntheseus.search.algorithms.breadth_first import AndOr_BreadthFirstSearch
from syntheseus.search.node_evaluation.common import (
    ConstantNodeEvaluator,
    ReactionModelLogProbCost,
    HasSolutionValueFunction,
    ReactionModelProbPolicy,
)
from syntheseus.search.analysis.route_extraction import iter_routes_time_order
from syntheseus.search.analysis.solution_time import get_first_solution_time
from syntheseus.search.visualization import visualize_andor, visualize_molset

import time
from pprint import pprint

print("✅ 导入成功！")

## 2. 单步反应模型

Syntheseus 支持多种最先进的单步逆合成模型。每个模型都有统一的接口，方便切换和比较。

### 2.1 可用模型列表

| 模型名称 | 特点 | 适用场景 |
|---------|------|----------|
| **LocalRetro** | 基于模板的局部逆合成 | 快速、准确 |
| **Chemformer** | Transformer架构 | 新颖反应 |
| **MEGAN** | 图编辑模型 | 平衡性能 |
| **MHNreact** | 分子超图网络 | 复杂反应 |
| **Graph2Edits** | 图到编辑模型 | 灵活 |
| **RetroKNN** | 基于检索 | 可解释性 |
| **RootAligned** | 根对齐模板 | 通用 |

### 2.2 初始化单步模型

In [ ]:
# 示例：初始化 LocalRetro 模型
# 如果未指定 model_dir，会自动下载并缓存预训练模型
model = LocalRetroModel(
    use_cache=True,  # 启用缓存以提高性能
    default_num_results=50  # 每次预测返回的反应数量
)

print(f"模型缓存路径: {model.model_dir}")
print("✅ 模型初始化完成！")

### 2.3 测试单步模型

In [ ]:
# 创建测试分子（联苯化合物）
test_mol = Molecule("Cc1ccc(-c2ccc(C)cc2)cc1")

# 获取逆合成预测
[results] = model([test_mol], num_results=5)

print(f"\n预测到 {len(results)} 个反应：\n")
for idx, prediction in enumerate(results, 1):
    reactants_str = " + ".join([mol.smiles for mol in prediction.reactants])
    prob = prediction.metadata.get('probability', 'N/A')
    print(f"{idx}. {reactants_str}")
    print(f"   概率: {prob}\n")

### 2.4 比较不同模型（可选）

In [ ]:
# 比较多个模型的预测结果
models_to_compare = {
    "LocalRetro": LocalRetroModel(),
    # 取消注释以比较更多模型
    # "Chemformer": ChemformerModel(),
    # "MEGAN": MEGANModel(),
}

print("\n各模型的首选反应预测：\n")
for model_name, model_obj in models_to_compare.items():
    [results] = model_obj([test_mol], num_results=5)
    if results:
        top_prediction = " + ".join([mol.smiles for mol in results[0].reactants])
        print(f"{model_name:15s}: {top_prediction}")

## 3. 原料库配置

原料库定义了哪些分子是可购买的起始物料。Syntheseus 提供了灵活的方式来管理原料库。

### 3.1 创建简单原料库

In [ ]:
# 方法1: 直接从 SMILES 列表创建
simple_inventory = SmilesListInventory(
    smiles_list=[
        "Cc1ccc(B(O)O)cc1",      # 4-甲基苯硼酸
        "Cc1ccc(Br)cc1",          # 4-溴甲苯
        "Cc1ccc(I)cc1",           # 4-碘甲苯
        "O=Cc1ccc(I)cc1",         # 4-碘苯甲醛
    ],
    canonicalize=True  # 自动规范化 SMILES
)

print(f"原料库大小: {len(simple_inventory)} 个分子")

# 测试分子是否可购买
test_purchasable = Molecule("Cc1ccc(Br)cc1")
test_not_purchasable = Molecule("Cc1ccc(-c2ccccc2)cc1")

print(f"\n{test_purchasable.smiles} 是否可购买: {simple_inventory.is_purchasable(test_purchasable)}")
print(f"{test_not_purchasable.smiles} 是否可购买: {simple_inventory.is_purchasable(test_not_purchasable)}")

### 3.2 从文件加载原料库

In [ ]:
# 创建示例原料库文件
inventory_file = "building_blocks.txt"

with open(inventory_file, 'w') as f:
    f.write("\n".join([
        "Cc1ccc(B(O)O)cc1",
        "Cc1ccc(Br)cc1",
        "Cc1ccc(I)cc1",
        "O=Cc1ccc(I)cc1",
        "Brc1ccccc1",
        "Ic1ccccc1",
        "B(O)(O)c1ccccc1",
    ]))

# 方法2: 从文件加载
inventory = SmilesListInventory.load_from_file(
    inventory_file,
    canonicalize=True
)

print(f"✅ 从文件加载了 {len(inventory)} 个建构块")
print("\n原料库中的分子:")
for mol in list(inventory.to_purchasable_mols())[:5]:
    print(f"  - {mol.smiles}")

## 4. 搜索算法配置

Syntheseus 支持多种搜索算法，每种都有不同的特点和适用场景。

### 4.1 Retro* 算法

Retro* 是一种最优优先搜索算法，适合寻找最短或最低成本的合成路径。

In [ ]:
# 配置 Retro* 算法
from syntheseus.search.algorithms.best_first import retro_star

# 1. 分子成本函数（可购买=0，不可购买=∞）
or_node_cost_fn = retro_star.MolIsPurchasableCost()

# 2. 反应成本函数（基于模型预测概率）
and_node_cost_fn = ReactionModelLogProbCost(normalize=False)

# 3. 启发式函数（值函数）
value_function = ConstantNodeEvaluator(0.0)  # Retro*-0 (最乐观)

# 创建 Retro* 搜索算法
retro_star_alg = RetroStarSearch(
    reaction_model=model,
    mol_inventory=inventory,
    or_node_cost_fn=or_node_cost_fn,
    and_node_cost_fn=and_node_cost_fn,
    value_function=value_function,
    limit_iterations=100,              # 最大迭代次数
    limit_reaction_model_calls=100,    # 最大模型调用次数
    time_limit_s=60.0,                 # 最大运行时间（秒）
    max_expansion_depth=10,            # 最大展开深度
)

print("✅ Retro* 算法配置完成")

### 4.2 MCTS 算法

蒙特卡洛树搜索（MCTS）是一种基于采样的算法，适合探索多样化的合成路径。

In [ ]:
# 配置 MCTS 算法
from syntheseus.search.algorithms.mcts.base import pucb_bound

# 1. 奖励函数（找到解=1，否则=0）
reward_function = HasSolutionValueFunction()

# 2. 值函数（用于估计未展开节点）
mcts_value_function = ConstantNodeEvaluator(0.5)

# 3. 策略函数（基于模型概率）
policy = ReactionModelProbPolicy(normalize=True, temperature=3.0)

# 创建 MCTS 算法
mcts_alg = MolSetMCTS(
    reaction_model=model,
    mol_inventory=inventory,
    reward_function=reward_function,
    value_function=mcts_value_function,
    policy=policy,
    bound_constant=100.0,
    bound_function=pucb_bound,
    limit_iterations=100,
    limit_reaction_model_calls=100,
    time_limit_s=60.0,
    max_expansion_depth=20,
)

print("✅ MCTS 算法配置完成")

### 4.3 广度优先搜索

简单但有效的基线算法。

In [ ]:
# 配置广度优先搜索
bfs_alg = AndOr_BreadthFirstSearch(
    reaction_model=model,
    mol_inventory=inventory,
    limit_iterations=100,
    limit_reaction_model_calls=100,
    time_limit_s=60.0,
)

print("✅ 广度优先搜索配置完成")

## 5. 运行多步搜索

现在我们可以对目标分子运行搜索了！

### 5.1 定义目标分子

In [ ]:
# 定义目标分子（联苯化合物）
target = Molecule("Cc1ccc(-c2ccc(C)cc2)cc1")

print(f"目标分子: {target.smiles}")
print(f"是否在原料库中: {inventory.is_purchasable(target)}")

### 5.2 使用 Retro* 搜索

In [ ]:
print("\n🔍 开始 Retro* 搜索...\n")
start_time = time.time()

# 运行搜索
retro_star_alg.reset()  # 重置算法状态
retro_star_graph, _ = retro_star_alg.run_from_mol(target)

retro_star_time = time.time() - start_time

print(f"✅ Retro* 搜索完成！")
print(f"   运行时间: {retro_star_time:.2f} 秒")
print(f"   探索的节点数: {len(retro_star_graph)}")
print(f"   模型调用次数: {model.num_calls()}")

### 5.3 使用 MCTS 搜索

In [ ]:
print("\n🔍 开始 MCTS 搜索...\n")
start_time = time.time()

# 重新初始化模型以清除缓存（公平比较）
model = LocalRetroModel(use_cache=True, default_num_results=50)
mcts_alg = MolSetMCTS(
    reaction_model=model,
    mol_inventory=inventory,
    reward_function=reward_function,
    value_function=mcts_value_function,
    policy=policy,
    bound_constant=100.0,
    bound_function=pucb_bound,
    limit_iterations=100,
    limit_reaction_model_calls=100,
    time_limit_s=60.0,
)

mcts_alg.reset()
mcts_graph, _ = mcts_alg.run_from_mol(target)

mcts_time = time.time() - start_time

print(f"✅ MCTS 搜索完成！")
print(f"   运行时间: {mcts_time:.2f} 秒")
print(f"   探索的节点数: {len(mcts_graph)}")
print(f"   模型调用次数: {model.num_calls()}")

## 6. 结果评测与分析

现在我们来分析和比较搜索结果。

### 6.1 提取合成路径

In [ ]:
# 从 Retro* 图中提取路径
retro_star_routes = list(iter_routes_time_order(retro_star_graph, max_routes=10))
print(f"Retro* 找到 {len(retro_star_routes)} 条合成路径")

# 从 MCTS 图中提取路径
from syntheseus.search.analysis.route_extraction import iter_routes_cost_order
from syntheseus.search.graph.and_or import AndNode
from syntheseus.search.graph.molset import MolSetNode

# 为成本提取设置路径成本
for node in mcts_graph.nodes():
    if isinstance(node, MolSetNode):
        node.data["route_cost"] = 1.0
    else:
        node.data["route_cost"] = 0.0

mcts_routes = list(iter_routes_cost_order(mcts_graph, max_routes=10))
print(f"MCTS 找到 {len(mcts_routes)} 条合成路径")

### 6.2 分析路径统计

In [ ]:
def count_reactions_in_route(route, graph):
    """统计路径中的反应数量"""
    if isinstance(graph, type(retro_star_graph)):
        return len([n for n in route if isinstance(n, AndNode)])
    else:
        return len([n for n in route if isinstance(n, MolSetNode)])

print("\n📊 路径统计:\n")

if retro_star_routes:
    print("Retro* 路径:")
    for idx, route in enumerate(retro_star_routes[:5], 1):
        num_rxns = count_reactions_in_route(route, retro_star_graph)
        print(f"  路径 {idx}: {num_rxns} 步反应")

if mcts_routes:
    print("\nMCTS 路径:")
    for idx, route in enumerate(mcts_routes[:5], 1):
        num_rxns = count_reactions_in_route(route, mcts_graph)
        print(f"  路径 {idx}: {num_rxns} 步反应")

### 6.3 计算首次求解时间

In [ ]:
# 设置时间度量（使用模型调用次数）
for node in retro_star_graph.nodes():
    node.data["analysis_time"] = node.data["num_calls_rxn_model"]

for node in mcts_graph.nodes():
    node.data["analysis_time"] = node.data["num_calls_rxn_model"]

# 计算首次求解时间
retro_star_soln_time = get_first_solution_time(retro_star_graph)
mcts_soln_time = get_first_solution_time(mcts_graph)

print("\n⏱️  首次求解时间（模型调用次数）:\n")
print(f"  Retro*: {retro_star_soln_time}")
print(f"  MCTS:   {mcts_soln_time}")

### 6.4 可视化合成路径

In [ ]:
# 可视化第一条 Retro* 路径
if retro_star_routes:
    visualize_andor(
        retro_star_graph,
        filename="retro_star_route_1.pdf",
        nodes=retro_star_routes[0]
    )
    print("✅ Retro* 路径已保存到 retro_star_route_1.pdf")

# 可视化第一条 MCTS 路径
if mcts_routes:
    visualize_molset(
        mcts_graph,
        filename="mcts_route_1.pdf",
        nodes=mcts_routes[0]
    )
    print("✅ MCTS 路径已保存到 mcts_route_1.pdf")

### 6.5 算法性能对比

In [ ]:
import pandas as pd

# 创建比较表格
comparison_data = {
    "算法": ["Retro*", "MCTS"],
    "运行时间(秒)": [retro_star_time, mcts_time],
    "探索节点数": [len(retro_star_graph), len(mcts_graph)],
    "找到路径数": [len(retro_star_routes), len(mcts_routes)],
    "首次求解时间": [retro_star_soln_time, mcts_soln_time],
}

df = pd.DataFrame(comparison_data)
print("\n📈 算法性能对比:\n")
print(df.to_string(index=False))

## 7. 高级功能

### 7.1 批量处理多个目标分子

In [ ]:
# 定义多个目标分子
targets = [
    Molecule("Cc1ccc(-c2ccc(C)cc2)cc1"),  # 联苯
    Molecule("COc1ccc(-c2ccc(OC)cc2)cc1"),  # 二甲氧基联苯
]

print("\n🔄 批量处理多个目标分子...\n")

batch_results = []
for idx, target_mol in enumerate(targets, 1):
    print(f"处理目标 {idx}: {target_mol.smiles}")
    
    # 重置并运行搜索
    model = LocalRetroModel(use_cache=True, default_num_results=50)
    bfs_alg = AndOr_BreadthFirstSearch(
        reaction_model=model,
        mol_inventory=inventory,
        limit_iterations=50,
        time_limit_s=30.0,
    )
    
    bfs_alg.reset()
    graph, _ = bfs_alg.run_from_mol(target_mol)
    
    # 提取路径
    routes = list(iter_routes_time_order(graph, max_routes=5))
    
    batch_results.append({
        "smiles": target_mol.smiles,
        "nodes": len(graph),
        "routes": len(routes),
        "solved": len(routes) > 0
    })
    
    print(f"  ✓ 探索节点: {len(graph)}, 找到路径: {len(routes)}\n")

# 显示结果
batch_df = pd.DataFrame(batch_results)
print("\n批量处理结果:")
print(batch_df.to_string(index=False))

### 7.2 自定义评估函数

你可以定义自己的评估函数来指导搜索。

In [ ]:
from syntheseus.search.node_evaluation.base import NoCacheNodeEvaluator
from syntheseus.search.graph.and_or import OrNode
from typing import Sequence, Optional

class CustomValueFunction(NoCacheNodeEvaluator):
    """自定义值函数示例：偏好较小的分子"""
    
    def _evaluate_nodes(
        self,
        nodes: Sequence[OrNode],
        graph=None,
    ) -> list[float]:
        values = []
        for node in nodes:
            # 基于分子大小的启发式
            num_atoms = node.mol.rdkit_mol.GetNumAtoms()
            # 较小的分子给予较低的估计成本
            value = num_atoms / 50.0
            values.append(value)
        return values

print("✅ 自定义值函数定义完成")
print("\n提示: 使用此函数替换默认的 ConstantNodeEvaluator 来指导搜索")

### 7.3 保存和加载搜索结果

In [ ]:
import pickle

# 保存搜索图
with open("retro_star_graph.pkl", "wb") as f:
    pickle.dump(retro_star_graph, f)

with open("mcts_graph.pkl", "wb") as f:
    pickle.dump(mcts_graph, f)

print("✅ 搜索结果已保存")

# 加载搜索图
with open("retro_star_graph.pkl", "rb") as f:
    loaded_graph = pickle.load(f)

print(f"✅ 加载的图包含 {len(loaded_graph)} 个节点")

### 7.4 使用命令行接口

Syntheseus 也提供了强大的命令行工具：

In [ ]:
%%bash
# 示例命令（需要在终端中运行）

# 运行搜索
python -m syntheseus.cli.search \
    search_target="Cc1ccc(-c2ccc(C)cc2)cc1" \
    inventory_smiles_file=building_blocks.txt \
    model_class=LocalRetro \
    search_algorithm=retro_star \
    time_limit_s=60 \
    results_dir=./search_results

# 评估单步模型
python -m syntheseus.cli.eval_single_step \
    model_class=LocalRetro \
    test_file=test_smiles.txt \
    output_file=predictions.pkl

## 总结

本教程展示了如何使用 Syntheseus 进行完整的多步逆合成规划，包括：

✅ **单步模型**: 多种最先进模型的使用和比较  
✅ **原料库**: 灵活的原料库管理  
✅ **搜索算法**: Retro*, MCTS, BFS 等算法的配置  
✅ **评测分析**: 路径提取、性能比较、可视化  
✅ **高级功能**: 批量处理、自定义函数、结果保存  

### 下一步

- 📖 阅读 [完整文档](https://microsoft.github.io/syntheseus/)
- 🔬 尝试不同的模型和算法组合
- 🎯 在你自己的数据集上测试
- 🚀 开发自定义的评估函数和启发式

### 参考资源

- **论文**: [Re-evaluating retrosynthesis algorithms with syntheseus](https://pubs.rsc.org/en/content/articlelanding/2024/fd/d4fd00093e)
- **GitHub**: [microsoft/syntheseus](https://github.com/microsoft/syntheseus)
- **文档**: [microsoft.github.io/syntheseus](https://microsoft.github.io/syntheseus/)